In [1]:
import os
import tqdm
import json
import shutil
import numpy as np
from collections import defaultdict, Counter
# from data_converter import *

def convert_coco_json(image_folder, annotation_file, output_folder, keep_fn=None, use_segments=True):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_image_folder = os.path.join(output_folder, "images")
    output_ann_folder = os.path.join(output_folder, "labels")
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_ann_folder, exist_ok=True)

    # Load coco annotation
    with open(annotation_file) as f:
        data = json.load(f)

    # Create image dict
    images = {"%g" % x["id"]: x for x in data["images"]}
    # Create image-annotations dict
    imgToAnns = defaultdict(list)
    for ann in data["annotations"]:
        imgToAnns[ann["image_id"]].append(ann)

    label_counter = Counter()
    # Write labels file
    # for img_id, anns in tqdm.tqdm(imgToAnns.items(), desc=f"Annotations {annotation_file}"):
    for img_id, anns in imgToAnns.items():
        img = images["%g" % img_id]
        h, w, f = img["height"], img["width"], img["file_name"]
        if keep_fn is not None and not keep_fn(f):
            continue

        bboxes = []
        segments = []
        for ann in anns:
            if ann["iscrowd"]:
                continue
            # The COCO box format is [top left x, top left y, width, height]
            box = np.array(ann["bbox"], dtype=np.float64)
            box[:2] += box[2:] / 2  # xy top-left corner to center
            box[[0, 2]] /= w  # normalize x
            box[[1, 3]] /= h  # normalize y
            if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
                continue

            cls = ann["category_id"] - 1
            label_counter[cls] += 1
            box = [cls] + box.tolist()
            if box not in bboxes:
                bboxes.append(box)
            # Segments
            if use_segments:
                if len(ann["segmentation"]) > 1:
                    s = merge_multi_segment(ann["segmentation"])
                    s = (np.concatenate(s, axis=0) / np.array([w, h])).reshape(-1).tolist()
                else:
                    s = [j for i in ann["segmentation"] for j in i]  # all segments concatenated
                    s = (np.array(s).reshape(-1, 2) / np.array([w, h])).reshape(-1).tolist()
                s = [cls] + s
                if s not in segments:
                    segments.append(s)

        # Write
        src_img_path = os.path.join(image_folder, f)
        new_img_path = os.path.join(output_image_folder, f)
        new_ann_path = os.path.join(output_ann_folder, f"{os.path.splitext(f)[0]}.txt")

        shutil.copy(src_img_path, new_img_path)   
        with open(new_ann_path, "w") as file:
            for i in range(len(bboxes)):
                line = (*(segments[i] if use_segments else bboxes[i]),)  # cls, box or segments
                file.write(("%g " * len(line)).rstrip() % line + "\n")
        
    return label_counter


In [2]:
output_folder_train = "./data_yolo/train"
label_counter_train = convert_coco_json(
    image_folder="./data_coco/images/livecell_train_val_images", 
    annotation_file="./data_coco/livecell_coco_train.json", 
    output_folder=output_folder_train,
    use_segments=True
)
print(f"Labels in train dataset: {label_counter_train}")

output_folder_val = "./data_yolo/val"
label_counter_val = convert_coco_json(
    image_folder="./data_coco/images/livecell_train_val_images", 
    annotation_file="./data_coco/livecell_coco_val.json", 
    output_folder=output_folder_val,
    use_segments=True
)
print(f"Labels in val dataset: {label_counter_val}")

output_folder_test = "./data_yolo/test"
label_counter_test = convert_coco_json(
    image_folder="./data_coco/images/livecell_test_images", 
    annotation_file="./data_coco/livecell_coco_test.json", 
    output_folder=output_folder_test,
    use_segments=True
)
print(f"Labels in val dataset: {label_counter_test}")


Labels in train dataset: Counter({0: 1018576})
Labels in val dataset: Counter({0: 181610})
Labels in val dataset: Counter({0: 462261})


In [3]:
yaml_file = [
    f"train: {os.path.join(output_folder_train, 'images')}",
    f"val: {os.path.join(output_folder_val, 'images')}",
    f"test: {os.path.join(output_folder_test, 'images')}",
    f"nc: 1",
    f"names: {['cell']}",
]
print("\n".join(yaml_file))

with open("data_livecell.yaml", "w") as f:
    f.write("\n".join(yaml_file))

train: ./data_yolo/train/images
val: ./data_yolo/val/images
test: ./data_yolo/test/images
nc: 1
names: ['cell']


In [4]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8s-seg.pt')

results = model.train(
    data='./data_livecell.yaml', 
    project='ckpts',
    name='livecell',
    epochs=10,
    batch=48,
    imgsz=640,
    device='cuda:1,2,3', # [1],
    single_cls=False, # 
    label_smoothing=0.0,
    overlap_mask=True,
    mask_ratio=4,

    #Augmentation parameters
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    degrees=0.0,  # (float) image rotation (+/- deg)
    translate=0.1,  # (float) image translation (+/- fraction)
    scale=0.2,  # (float) image scale (+/- gain)
    shear=0.0,  # (float) image shear (+/- deg)
    perspective=0.0,  # (float) image perspective (+/- fraction), range 0-0.001
    flipud=0.5,  # (float) image flip up-down (probability)
    fliplr=0.5,  # (float) image flip left-right (probability)
    mosaic=1.0,  # (float) image mosaic (probability)
    mixup=0.0,  # (float) image mixup (probability)
    copy_paste=0.0,  # (float) segment copy-paste (probability)
)


Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:2 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:3 (Tesla V100-SXM2-32GB, 32501MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=./data_livecell.yaml, epochs=10, time=None, patience=100, batch=48, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:1,2,3, workers=8, project=ckpts, name=livecell, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, 

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:2 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:3 (Tesla V100-SXM2-32GB, 32501MiB)
Overriding model.yaml nc=80 with nc=1
Transferred 411/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/train/labels.cache... 3188 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3188/3188 [00:00<?, ?it/s]


train: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/train/images/BV2_Phase_D4_2_02d12h00m_4.tif: 2 duplicate labels removed
train: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/train/images/BV2_Phase_D4_2_02d16h00m_4.tif: 1 duplicate labels removed


...

                   all        568     181540      0.819      0.518      0.652      0.414        0.8        0.5      0.622      0.299

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      10/10      25.6G      1.515      2.501     0.7182      1.048       1643        640: 100%|██████████| 67/67 [00:42<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:13<00:00,  1.32it/s]


                   all        568     181540      0.839      0.524      0.661      0.421      0.819      0.507      0.634      0.312

10 epochs completed in 0.184 hours.
Optimizer stripped from ckpts/livecell/weights/last.pt, 23.8MB
Optimizer stripped from ckpts/livecell/weights/best.pt, 23.8MB

Validating ckpts/livecell/weights/best.pt...
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:2 (Tesla V100-SXM2-32GB, 32501MiB)
                                                           CUDA:3 (Tesla V100-SXM2-32GB, 32501MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [01:25<00:00,  4.75s/it]


                   all        568     181540      0.839      0.521      0.658      0.421      0.823      0.506      0.635      0.316
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to ckpts/livecell


In [5]:
# Evaluate the Overall model performance on the validation set
# model = YOLO('./ckpts/livecell/weights/best.pt')
print("Evaluate on validation dataset:")
val_stats = model.val(data='./data_livecell.yaml', device='cuda:1', split='val')

print("Evaluate on testing dataset:")
test_stats = model.val(data='./data_livecell.yaml', device='cuda:1', split='test')


Evaluate on validation dataset:
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


val: Scanning /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/val/labels.cache... 568 images, 0 backgrounds, 0 corrupt: 100%|██████████| 568/56
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/12 [00:00<?/project/DPDS/Xiao_lab/shared/CondaEnv/hd_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [01:28<


                   all        568     181540      0.839      0.521      0.658      0.423      0.818      0.503      0.631      0.306
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 12.0ms postprocess per image
Results saved to ckpts/livecell2
Evaluate on testing dataset:
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)


val: Scanning /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/labels.cache... 1512 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1512

val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/Huh7_Phase_A12_2_03d04h00m_2.tif: 1 duplicate labels removed
val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/MCF7_Phase_H4_1_00d00h00m_2.tif: 1 duplicate labels removed
val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/SHSY5Y_Phase_A10_2_03d08h00m_2.tif: 1 duplicate labels removed
val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/SKOV3_Phase_E4_2_02d16h00m_3.tif: 2 duplicate labels removed
val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/SKOV3_Phase_E4_2_02d20h00m_3.tif: 1 duplicate labels removed
val: WARNING ⚠️ /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/data_yolo/test/images/SKOV3_Phase_E4_2_03d00h00m_4.tif: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   9%|▉         | 3/32 [00:41<0

WARNING ⚠️ NMS time limit 4.400s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [02:31<


                   all       1512     453041       0.84      0.531      0.661      0.422      0.824      0.515      0.636      0.317
Speed: 0.1ms preprocess, 4.4ms inference, 0.0ms loss, 5.6ms postprocess per image
Results saved to ckpts/livecell3


In [6]:
# Evaluate each categories on the testing dataset
# model = YOLO('./ckpts/livecell/weights/best.pt')

MARKERS = ['A172', 'BT474', 'BV2', 'Huh7', 'MCF7', 'SHSY5Y', 'SkBr3', 'SKOV3']

output_folder_splits = "./data_yolo_split"
label_counters = {
    marker: convert_coco_json(
        image_folder="./data_coco/images/livecell_test_images", 
        annotation_file="./data_coco/livecell_coco_test.json", 
        output_folder=os.path.join(output_folder_splits, marker),
        keep_fn=lambda x: x.startswith(marker),
        use_segments=True
    )[0] for marker in MARKERS
}
# label_counters = {'A172': 32978, 'BT474': 32585, 'BV2': 89970, 'Huh7': 12552, 'MCF7': 96914, 'SHSY5Y': 75988, 'SkBr3': 65757, 'SKOV3': 55517}
print(label_counters)
print("================")

yaml_file_split = [
    f"{marker}: {os.path.join(output_folder_splits, marker, 'images')}"
    for marker in MARKERS
] + yaml_file
print("\n".join(yaml_file_split))

with open("data_livecell_test_split.yaml", "w") as f:
    f.write("\n".join(yaml_file_split))

print("================")
print("Evaluate on testing dataset for each categories:")

test_stats_split = {}
for marker in MARKERS:
    print("================")
    print(f"marker: {marker}")
    stats = model.val(data='./data_livecell_test_split.yaml', device='cuda:1', split=marker)
    test_stats_split[marker] = stats
    print("================")


{'A172': 32978, 'BT474': 32585, 'BV2': 89970, 'Huh7': 12552, 'MCF7': 96914, 'SHSY5Y': 75988, 'SkBr3': 65757, 'SKOV3': 55517}
A172: ./data_yolo_split/A172/images
BT474: ./data_yolo_split/BT474/images
BV2: ./data_yolo_split/BV2/images
Huh7: ./data_yolo_split/Huh7/images
MCF7: ./data_yolo_split/MCF7/images
SHSY5Y: ./data_yolo_split/SHSY5Y/images
SkBr3: ./data_yolo_split/SkBr3/images
SKOV3: ./data_yolo_split/SKOV3/images
train: ./data_yolo/train/images
val: ./data_yolo/val/images
test: ./data_yolo/test/images
nc: 1
names: ['cell']
Evaluate on testing dataset for each categories:
marker: A172
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)


val: Scanning data_yolo_split/A172/labels.cache... 152 images, 0 backgrounds, 0 corrupt: 100%|██████████| 152/152 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:40<00


                   all        152      32978       0.76      0.657      0.709        0.4      0.779      0.666      0.721      0.369
Speed: 0.2ms preprocess, 15.9ms inference, 0.0ms loss, 15.0ms postprocess per image
Results saved to ckpts/livecell4
marker: BT474
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)


val: Scanning data_yolo_split/BT474/labels.cache... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:50<00


                   all        168      32584      0.706      0.642      0.711      0.409      0.694      0.615      0.679      0.321
Speed: 0.4ms preprocess, 11.0ms inference, 0.0ms loss, 25.5ms postprocess per image
Results saved to ckpts/livecell5
marker: BV2
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)


...

                   all        176      65757      0.982      0.712      0.842      0.633      0.962      0.697      0.826      0.475
Speed: 0.1ms preprocess, 5.9ms inference, 0.0ms loss, 23.9ms postprocess per image
Results saved to ckpts/livecell10
marker: SKOV3
Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CUDA:1 (Tesla V100-SXM2-32GB, 32501MiB)


val: Scanning data_yolo_split/SKOV3/labels.cache... 304 images, 0 backgrounds, 0 corrupt: 100%|██████████| 304/304 [00:00<?, ?it/s]

val: WARNING ⚠️ data_yolo_split/SKOV3/images/SKOV3_Phase_E4_2_02d16h00m_3.tif: 2 duplicate labels removed
val: WARNING ⚠️ data_yolo_split/SKOV3/images/SKOV3_Phase_E4_2_02d20h00m_3.tif: 1 duplicate labels removed
val: WARNING ⚠️ data_yolo_split/SKOV3/images/SKOV3_Phase_E4_2_03d00h00m_4.tif: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:45<00


                   all        304      47361      0.878      0.807      0.869      0.558      0.885      0.805      0.865      0.502
Speed: 0.8ms preprocess, 7.0ms inference, 0.0ms loss, 8.6ms postprocess per image
Results saved to ckpts/livecell11


In [7]:
## summarize testing dataset performance into dataframe
import pandas as pd
from IPython.display import display, HTML

test_stats_split['livecell'] = test_stats  # add overall performance

summary_table = [
    {'Dataset': key, 
     'Box Precision': test_stats_split[key].box.mp,
     'Box Recall': test_stats_split[key].box.mr,
     'Box mAP50': test_stats_split[key].box.map50, 
     'Mask Precision': test_stats_split[key].seg.mp,
     'Mask Recall': test_stats_split[key].seg.mr,
     'Mask mAP50': test_stats_split[key].seg.map50
    }
    for key in ['livecell'] + MARKERS
]
display(pd.DataFrame(summary_table))

,Dataset,Box Precision,Box Recall,Box mAP50,Mask Precision,Mask Recall,Mask mAP50
0,livecell,0.839859,0.531477,0.660601,0.824087,0.514530,0.636406
1,A172,0.760340,0.656983,0.709321,0.779101,0.665595,0.720566
2,BT474,0.706422,0.641849,0.711405,0.694499,0.615351,0.678568
3,BV2,0.969109,0.383378,0.627921,0.902256,0.356257,0.582586
4,Huh7,0.814017,0.748609,0.833508,0.796128,0.735483,0.801921
5,MCF7,0.856477,0.415225,0.617170,0.830692,0.400045,0.592256
6,SHSY5Y,0.694840,0.407860,0.535170,0.674894,0.383726,0.506154
7,SkBr3,0.981506,0.711574,0.841601,0.962302,0.696808,0.826461
8,SKOV3,0.878324,0.806580,0.869130,0.884575,0.805093,0.865365


In [8]:
# Visualize instance segmentation on testing images
import random
import matplotlib.pyplot as plt
random.seed(123)

test_images = [os.path.join(os.path.join(output_folder_test, "images"), file) 
               for file in os.listdir(os.path.join(output_folder_test, "images"))]
display_images = random.sample(test_images, 10)
results = model(display_images)

# Show the results
for r in results:
    fig, axes = plt.subplots(1, 1, figsize=(18, 18))
    plt.imshow(r.plot(font_size=10, labels=False)[...,::-1])  # plot a BGR numpy array of predictions
    plt.show()
    # im = Image.fromarray(im[..., ::-1])  # RGB PIL image
    # im_array.show()  # show image



0: 480x640 300 cells, 5.9ms
1: 480x640 195 cells, 5.9ms
2: 480x640 132 cells, 5.9ms
3: 480x640 300 cells, 5.9ms
4: 480x640 212 cells, 5.9ms
5: 480x640 57 cells, 5.9ms
6: 480x640 176 cells, 5.9ms
7: 480x640 185 cells, 5.9ms
8: 480x640 300 cells, 5.9ms
9: 480x640 50 cells, 5.9ms
Speed: 1.9ms preprocess, 5.9ms inference, 12.8ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

...

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

<Figure size 1800x1800 with 1 Axes>

In [9]:
## Export the model to ONNX format
success = model.export(format='onnx', device='cpu')

Ultralytics YOLOv8.2.10 🚀 Python-3.11.5 torch-2.1.0+cu118 CPU (Intel Xeon Gold 6240 2.60GHz)

PyTorch: starting from 'ckpts/livecell/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (22.7 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.0s, saved as 'ckpts/livecell/weights/best.onnx' (45.2 MB)

Export complete (1.4s)
Results saved to /project/DPDS/Xiao_lab/shared/deep_learning_SW_RR/livecell/ckpts/livecell/weights
Predict:         yolo predict task=segment model=ckpts/livecell/weights/best.onnx imgsz=640  
Validate:        yolo val task=segment model=ckpts/livecell/weights/best.onnx imgsz=640 data=./data_livecell.yaml  
Visualize:       https://netron.app
